# DEMO: Distributing your experiment as a pip installable package

The aim of this demo is to show how we can turn our experiment into a Python package, distribute it online and automatically test it using GitHub actions. Let's start by showing the finished product. We can use pip to install a package directly from github by typing `pip install git+` followed by the url of the repository. Executing the cell below, will install the experiment int your current Python environment. 

In [ ]:
%pip install git+https://github.com/OleBialas/posner_task.git

The experiment comes with a command line interface (CLI). 
Lets call the CLI's help:

In [ ]:
!posner -h

The help documentation tells us that this program expects the path to a JSON file contaning documentation. Let's quickly create one:

In [4]:
import json
config = {
        "root_dir": "",
        "fix_dur": 0.5,
        "cue_dur": 0.5,
        "fix_radius": 0.025,
        "fix_color": "white",
        "stim_radius": 0.05,
        "stim_color":"red",
        "n_blocks": 2,
        "n_trials": 9,
        "p_valid": 0.8,
        "pos": {"left": (-0.5, 0), "right": (0.5, 0)},
    }
with open("config.json", "w") as f:
    json.dump(config, f, indent=4)

Now we can run the experiment for subject 1 with the created config by running the cell below

In [ ]:
!posner 1 config.json

The CLI also has a `--test` flag that runs an automated test of the experiment. Let's create a new config with very short wait times so the test can run really fast. Let's also change the number of blocks to **5** and the number of trials per block to **20**. We'll call our new config file `test_config.json`

In [ ]:
config = {
        "root_dir": "",
        "fix_dur": 0.01,
        "cue_dur": 0.01,
        "fix_radius": 0.025,
        "fix_color": "white",
        "stim_radius": 0.05,
        "stim_color":"red",
        "n_blocks": 5,
        "n_trials": 20,
        "p_valid": 0.8,
        "pos": {"left": (-0.5, 0), "right": (0.5, 0)},
    }
with open("test_config.json", "w") as f:
        json.dump(config, f, indent=4)

In [ ]:
!posner 1 test_config.json --overwrite --test

We can use the data generated by this test to do a sanity check on our data analysis pipeline. First, we load all blocks and concatenate them into one data frame

In [ ]:
from pathlib import Path
import pandas as pd

blocks = []
for f in Path('data/sub-01').glob('block_*.csv'):
    blocks.append(pd.read_csv(f))
df = pd.concat(blocks)
df.head()

Then, for example, we may do a t-test on the difference in reaction times between trials with valid and invalid cues

In [ ]:
from scipy.stats import ttest_ind
rt_invalid = df[df.valid==False].response_time
rt_valid = df[df.valid==True].response_time

rt_valid = rt_valid.sample(n=len(rt_invalid))  # equalize number of trials
ttest_ind(rt_invalid, rt_valid)

The program uses a library called [Pydantic](https://docs.pydantic.dev/latest/) to validate the content of the configuration file. Upon loading, every field of the configuration is checked for its data type. Pydantic automatically raises an error when one of the field does not contain the desired type. This is very useful because it prevents our experiment from crashing due to an invalid configuration file.

Let's test this out by importing the experiment's `Config` class and creating a configuration with an invalid parameter

In [ ]:
from posner.experiment import Config
config["n_trials"] = "n"
Config(**config)

The module also contains a folder of test functions which can be executed using Pytest

In [ ]:
!pytest

In order to turn the experiment into a package, we have to provie the necessary information in a `pyproject.toml` file.
What's especially important here is the line `posner = "posner.experiment:main_cli"` because it determines how the program can be called from the command line.

```
[build-system]
requires = ["setuptools>=61.0"]
build-backend = "setuptools.build_meta"

[project.scripts]
posner = "posner.experiment:main_cli"

[project]
name = "posner"
version = "0.0.1"
authors = [
  { name="Ole Bialas", email="ole.bialas@posteo.de" },
]
description = "Psychopy implementation of the Posner attention cueing task."
readme = "README.md"
classifiers = [
    "Programming Language :: Python :: 3",
    "License :: OSI Approved :: MIT License",
    "Operating System :: OS Independent",
]
dependencies = ["psychopy"]

[project.urls]
Homepage = "https://github.com/OleBialas/posner_task"
Issues = "https://github.com/OleBialas/posner_task/issues"
```

Once our folder contains a `pyproject.toml` file we can use pip to build and install it as a package from the local files

In [ ]:
%pip install .

However, we can also upload it to GitHub and install it from anywhere!
Another useful thing about GitHub is that it allow us to define automated workflows that build and test our package everytime we make a change.